<a href="https://colab.research.google.com/github/ProtossDragoon/CoMoLab/blob/master/CV/Classification/c3d_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Googlee

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Import Tensorflow (version 1.15)

In [ ]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf
assert tf.__version__.startswith('1')
print(tf.__version__)

import tensorflow.keras as keras

1.15.2


Model

In [ ]:
def c3d():
  pass

In [ ]:
from tensorflow.keras.layers import Conv2D as Conv2D
from tensorflow.keras.layers import Conv3D as Conv3D
from tensorflow.keras.layers import MaxPooling3D as MaxPool3D
from tensorflow.keras.layers import GlobalAveragePooling3D as GAP3D
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import Softmax, Reshape, Dropout
from tensorflow.keras.regularizers import l2

def c3d_model(nb_classes=None, input_shape=(16, 112, 112, 3), summary=True, include_top=True):
    """ 
    Return the Keras model of the network

    Args:
      nb_classes: include_top 이 True 일 때에만 필요합니다. 분류기에서 분류할 class 의 수를 입력합니다.
      input_shape: model 의 input shape 을 정의합니다.
      summary: 모델 구조의 요약을 출력합니다.
      include_top: 분류기까지 함께 return 합니다.
    
    Returns:
      c3d model
    """

    frame_per_batch = input_shape[0]
    weight_decay = 0.005

    inputs = keras.Input(shape=input_shape)
    # 실제 input shape : (batch, frame, h, w, ch)

    x = Conv3D(64, (3,3,3), padding='same', 
               activation='relu', kernel_regularizer=l2(weight_decay), 
               name='conv1')(inputs)
    x = BatchNorm()(x)           
    x = MaxPool3D((2,2,1), strides=(2,2,1), padding='same')(x)

    x = Conv3D(128, (3,3,3), strides=(1,1,1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay),
               name='conv2')(x)
    x = BatchNorm()(x)
    x = MaxPool3D((2,2,2), strides=(2,2,2), padding='same')(x)

    x = Conv3D(256, (3,3,3), strides=(1,1,1), padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay),
               name='conv3-1')(x)
    x = BatchNorm()(x)
    x = Conv3D(256, (3,3,3), strides=(1,1,1), padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay),
               name='conv3-2')(x)
    x = BatchNorm()(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)

    x = Conv3D(512, (3,3,3), strides=(1,1,1), padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay),
               name='conv4-1')(x)
    x = BatchNorm()(x)
    x = Conv3D(512, (3,3,3), strides=(1,1,1),padding='same',
               activation='relu',kernel_regularizer=l2(weight_decay),
               name='conv4-2')(x)
    x = BatchNorm()(x)
    x = MaxPool3D((2,2,2),strides=(2,2,2),padding='same')(x)

    x = GAP3D()(x)
    x = Reshape((1,1,512))(x)

    x = Conv2D(512 * frame_per_batch, (1,1), strides=(1,1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay),
               name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Conv2D(512 * frame_per_batch, (1,1), strides=(1,1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay),
               name='fc2')(x)               
    x = Dropout(0.5)(x)

    if include_top:
        print('Input -> 3D Conv (Backbone) -> 1x1 Conv (Classifier) -> Output')
        x = Conv2D(nb_classes, kernel_regularizer=l2(weight_decay))(x)
        x = Softmax()(x)
    else :
        print('Input -> 3D Conv (Backbone) -> Output')

    model = tf.keras.models.Model(inputs, x)

    if summary:
        model.summary()

    return model

keras.backend.clear_session()
model = c3d_model(include_top=False)

Input -> 3D Conv (Backbone) -> Output
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 112, 112, 3)] 0         
_________________________________________________________________
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 112, 112, 64)  256       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 56, 112, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 8, 56, 112, 128)   221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 56, 112, 128)   512       
_________________________________________________________________
max_pooling3d_1 (MaxPoo

In [ ]:
def lstm_classifier(backbone, summary=True):
    input = backbone.layers[0].input
    output = backbone.layers[-1].output
    
    x = Dropout(0.5)(output)
    model = tf.keras.models.Model(input, x)
    
    if summary:
        model.summary()
  
    return model

model = lstm_classifier(model)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 112, 112, 3)] 0         
_________________________________________________________________
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 112, 112, 64)  256       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 56, 112, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 8, 56, 112, 128)   221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 56, 112, 128)   512       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 28, 56, 128)    0   